# SQL 3 - задача на сложную аггрегацию.

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определённой дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Дана таблица **peas**:  

<table align="left">
	<tr>
		<th>Название атрибута</th>
		<th>Тип атрибута</th>
		<th>Смысловое значение</th>
 	</tr>
 	<tr>
  		<td>st_id</td>
   		<td>int</td>
		<td>ID ученика</td>
 	</tr>
	<tr>
  		<td>timest</td>
   		<td>timestamp</td>
		<td>Время решения карточки</td>
 	</tr>
	<tr>
  		<td>correct</td>
   		<td>bool</td>
		<td>Правильно ли решена горошина?</td>
 	</tr>
	<tr>
  		<td>subject</td>
   		<td>text</td>
		<td>Дисциплина, в которой находится горошина</td>
 	</tr>
</table> 

Таблица **studs**:  

<table align="left">
	<tr>
		<th>Название атрибута</th>
		<th>Тип атрибута</th>
		<th>Смысловое значение</th>
 	</tr>
 	<tr>
  		<td>st_id</td>
   		<td>int</td>
		<td>ID ученика</td>
 	</tr>
	<tr>
  		<td>test_grp</td>
   		<td>text</td>
		<td>Метка ученика в данном эксперименте</td>
 	</tr>
</table> 

Таблица **checks**:  

<table align="left">
	<tr>
		<th>Название атрибута</th>
		<th>Тип атрибута</th>
		<th>Смысловое значение</th>
 	</tr>
 	<tr>
  		<td>st_id</td>
   		<td>int</td>
		<td>ID ученика</td>
 	</tr>
	<tr>
  		<td>sale_time</td>
   		<td>timestamp</td>
		<td>Время покупки</td>
 	</tr>
 	<tr>
  		<td>subject</td>
   		<td>text</td>
		<td>Дисциплина, на которую приобрели полный доступ</td>
 	</tr>
	<tr>
  		<td>money</td>
   		<td>int</td>
		<td>Цена, по которой приобрели данный курс</td>
 	</tr>
</table> 

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:
- ARPU
- ARPAU
- CR в покупку
- СR активного пользователя в покупку
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

#### Решение


Решение задачи реализовано непосредственно в jupyter на языке sqlite3.

В начале создадим датасеты из условий для проверки создаваемого кода. 

In [1]:
# Импортируем библеотеки. Создаём подключение к БД.

import pandas as pd
import numpy as np
import sqlite3

con = sqlite3.connect('db')

In [2]:
# Создаём peas

df_peas = pd.DataFrame({
    'st_id':[
        1,1,1,1,1,
        2,2,2,2,2,
        3,3,3,3,3], 
    'timest':pd.to_datetime([
        '2021-03-01 00:23:00','2021-03-01 00:11:00','2021-03-01 00:36:00','2021-03-01 00:45:00','2021-03-01 00:02:00',
        '2021-01-01 00:01:00','2021-01-01 00:56:00','2021-01-01 00:23:00','2021-01-01 00:33:00','2021-01-01 00:27:00',
        '2021-03-01 00:17:00','2021-03-01 00:23:00','2021-03-01 00:36:00','2021-03-01 00:26:00','2021-03-01 00:55:00'],
        format='%Y-%m-%d %H:%M:%S'), 
    'correct':[
        'true','true','true','false','true',
        'true','true','false','true','true',
        'false','false','true','true','true'], 
    'subject':[
        'a','a','a','a','b',
        'a','b','b','b','b',
        'a','c','c','c','c']
})

df_peas

,st_id,timest,correct,subject
0,1,2021-03-01 00:23:00,true,a
1,1,2021-03-01 00:11:00,true,a
2,1,2021-03-01 00:36:00,true,a
3,1,2021-03-01 00:45:00,false,a
4,1,2021-03-01 00:02:00,true,b
5,2,2021-01-01 00:01:00,true,a
6,2,2021-01-01 00:56:00,true,b
7,2,2021-01-01 00:23:00,false,b
8,2,2021-01-01 00:33:00,true,b
9,2,2021-01-01 00:27:00,true,b


In [3]:
# Создаём studs

df_studs = pd.DataFrame({
    'st_id':[
        1,2,3], 
    'test_grp':[
        'A','A','B']
})

df_studs

,st_id,test_grp
0,1,A
1,2,A
2,3,B


In [4]:
# Создаём checks

df_checks = pd.DataFrame({
    'st_id':[
        1,2,3], 
    'sale_time':pd.to_datetime([
        '2021-03-01 00:23:00','2021-03-05 00:11:00','2021-03-11 00:36:00'],
        format='%Y-%m-%d %H:%M:%S'), 
    'subject':[
        'a','a','c'], 
    'money':[
        '50','200','300']
})

df_checks

,st_id,sale_time,subject,money
0,1,2021-03-01 00:23:00,a,50
1,2,2021-03-05 00:11:00,a,200
2,3,2021-03-11 00:36:00,c,300


In [5]:
# Заливаем таблицу в sql.

df_peas.to_sql('peas_new', con, index=False, if_exists='replace')

df_studs.to_sql('studs', con, index=False, if_exists='replace')

df_checks.to_sql('checks', con, index=False, if_exists='replace')


3

**Переходим к SQL.**

Общая логика решения следующая:  
- ШАГ 1: объединить все таблицы вместе;
- ШАГ 2: расчитать активных пользователей;
- ШАГ 3: собрать агрегированную таблицу со всеми необходимыми данными;
- ШАГ 4: произвести расчеты:
    - ARPU, 
    - ARPAU,
    - CR в покупку, 
    - СR активного пользователя в покупку,
    - CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике.

In [6]:
# ШАГ 1: объединить все таблицы вместе.

# Объединяем все таблицы вместе через LEFT JOIN по id студента.
# Для удобства пометим id и subject из таблицы checks меткой "_buy".


sql = '''
SELECT
p.st_id, p.subject, s.test_grp, c.st_id AS st_id_buy, c.sale_time, c.subject AS subject_buy, c.money
FROM peas_new AS p
LEFT JOIN studs AS s ON p.st_id = s.st_id
LEFT JOIN checks AS c ON p.st_id = c.st_id
'''

In [7]:
pd.read_sql(sql, con)

,st_id,subject,test_grp,st_id_buy,sale_time,subject_buy,money
0,1,a,A,1,2021-03-01 00:23:00,a,50
1,1,a,A,1,2021-03-01 00:23:00,a,50
2,1,a,A,1,2021-03-01 00:23:00,a,50
3,1,a,A,1,2021-03-01 00:23:00,a,50
4,1,b,A,1,2021-03-01 00:23:00,a,50
5,2,a,A,2,2021-03-05 00:11:00,a,200
6,2,b,A,2,2021-03-05 00:11:00,a,200
7,2,b,A,2,2021-03-05 00:11:00,a,200
8,2,b,A,2,2021-03-05 00:11:00,a,200
9,2,b,A,2,2021-03-05 00:11:00,a,200


In [8]:
# ШАГ 2: расчитать активных пользователей.

# Предыдущий код уберем в подзапрос "general" с помощью with.

# Сделаем промежуточную таблицу для расчёта активных пользователей.
# Для этого создадим вспомогательный столбец "day", оставив в нём информацию только о дне.
# Для каждого значения даты суммируем кол-во правильно решенных горошин.
# Расчитаем с помощью условия case when кол-во правильно решенных горошин.
# Так же разметим с помощью условия case when активных пользователей (горошины/день >= 3).
# Сгруппируем данные по id, по предмету и по дню.

# В итоге получаем кол-во правильно решенных горошин в сутки для каждого пользователя в разрезе предмета.

# В рамках проверочного датасета я изменю требование к активному пользователю с 30 горошин/день на 3 горошины/день,
# но в итоговом коде вернусь к изначальному требованию. 


sql = '''
with general AS 
    (SELECT
    p.st_id, p.subject, s.test_grp, c.st_id AS st_id_buy, c.sale_time, c.subject AS subject_buy, c.money
    FROM peas_new AS p
    LEFT JOIN studs AS s ON p.st_id = s.st_id
    LEFT JOIN checks AS c ON p.st_id = c.st_id)

SELECT
p.st_id,
p.subject,
date(p.timest) AS day,
sum(CASE
    WHEN (p.correct = 'true' OR p.correct = 1)
    THEN 1 ELSE null
    END) AS sum_correct,
CASE
    WHEN (sum(CASE
                WHEN (p.correct = 'true' OR p.correct = 1)
                THEN 1 ELSE null
                END) >= 3)
    THEN 1 ELSE null
    END AS active_users
FROM peas_new as p
GROUP BY p.st_id, p.subject, day
'''

In [9]:
pd.read_sql(sql, con)

,st_id,subject,day,sum_correct,active_users
0,1,a,2021-03-01,3.0,1.0
1,1,b,2021-03-01,1.0,NaN
2,2,a,2021-01-01,1.0,NaN
3,2,b,2021-01-01,3.0,1.0
4,3,a,2021-03-01,NaN,NaN
5,3,c,2021-03-01,3.0,1.0


In [10]:
# ШАГ 3: собрать агрегированную таблицу со всеми необходимыми данными.

# Предыдущий код уберем в подзапрос "active_users" с помощью with.

# Соберем и структурируем все необходимые необходимые данные в одной таблице:
    # id студента,
    # тестовая группа (A или B),
    # предмет обучения,
    # кол-во студентов,
    # купленный предмет,
    # кол-во студентов купленного предмета (через условие case when, чтобы убрать задублирование),
    # кол-во активных студентов,
    # стоимость предмета  (через условие case when, чтобы убрать задублирование),
# Сгруппируем данные по id, по предмету обучения, по предмету покупки и по стоимости предмета.

# Основные данные возьмём из "general", данные по активным пользоватям из "active_users", приджойнив по id и subject.


sql = '''
with general AS 
    (SELECT
    p.st_id, p.subject, s.test_grp, c.st_id AS st_id_buy, c.sale_time, c.subject AS subject_buy, c.money
    FROM peas_new AS p
    LEFT JOIN studs AS s ON p.st_id = s.st_id
    LEFT JOIN checks AS c ON p.st_id = c.st_id),

active_users AS
    (SELECT
    p.st_id,
    p.subject,
    date(p.timest) AS day,
    sum(CASE
        WHEN (p.correct = 'true' OR p.correct = 1)
        THEN 1 ELSE null
        END) AS sum_correct,
    CASE
        WHEN (sum(CASE
                    WHEN (p.correct = 'true' OR p.correct = 1)
                    THEN 1 ELSE null
                    END) >= 3)
        THEN 1 ELSE null
        END AS active_users
    FROM peas_new as p
    GROUP BY p.st_id, p.subject, day)

SELECT
g.st_id,
g.test_grp,
g.subject,
COUNT(DISTINCT g.st_id) AS cnt_st_ALL,
g.subject_buy,
CASE
    WHEN g.subject = g.subject_buy
    THEN COUNT(DISTINCT g.st_id_buy) ELSE null 
    END AS cnt_st_BUY,
COUNT(DISTINCT au.active_users) AS cnt_st_ACTIVE,
CASE
    WHEN g.subject = g.subject_buy
    THEN g.money ELSE null 
    END AS money
FROM general AS g
LEFT JOIN active_users AS au ON g.st_id = au.st_id AND g.subject = au.subject
GROUP BY g.st_id, g.test_grp, g.subject, g.money
'''

In [11]:
pd.read_sql(sql, con)

,st_id,test_grp,subject,cnt_st_ALL,subject_buy,cnt_st_BUY,cnt_st_ACTIVE,money
0,1,A,a,1,a,1.0,1,50
1,1,A,b,1,a,NaN,0,None
2,2,A,a,1,a,1.0,0,200
3,2,A,b,1,a,NaN,1,None
4,3,B,a,1,c,NaN,0,None
5,3,B,c,1,c,1.0,1,300


In [12]:
# ШАГ 3: собрать агрегированную таблицу со всеми необходимыми данными.

# Предыдущий код уберем в подзапрос "agregated" с помощью with.

# Осталось схлопнуть все данные в удобный вид.
# Выводим все те же данные, что описаны выше, только суммируем в рамках своих групп.

# В итоге получаем агрегированные данные, с которыми уже легко дальше работать.


sql = '''
with general AS 
    (SELECT
    p.st_id, p.subject, s.test_grp, c.st_id AS st_id_buy, c.sale_time, c.subject AS subject_buy, c.money
    FROM peas_new AS p
    LEFT JOIN studs AS s ON p.st_id = s.st_id
    LEFT JOIN checks AS c ON p.st_id = c.st_id),

active_users AS
    (SELECT
    p.st_id,
    p.subject,
    date(p.timest) AS day,
    sum(CASE
        WHEN (p.correct = 'true' OR p.correct = 1)
        THEN 1 ELSE null
        END) AS sum_correct,
    CASE
        WHEN (sum(CASE
                    WHEN (p.correct = 'true' OR p.correct = 1)
                    THEN 1 ELSE null
                    END) >= 3)
        THEN 1 ELSE null
        END AS active_users
    FROM peas_new as p
    GROUP BY p.st_id, p.subject, day),

agregated AS 
    (SELECT
    g.st_id,
    g.test_grp,
    g.subject,
    COUNT(DISTINCT g.st_id) AS cnt_st_ALL,
    g.subject_buy,
    CASE
        WHEN g.subject = g.subject_buy
        THEN COUNT(DISTINCT g.st_id_buy) ELSE null 
        END AS cnt_st_BUY,
    COUNT(DISTINCT au.active_users) AS cnt_st_ACTIVE,
    CASE
        WHEN g.subject = g.subject_buy
        THEN g.money ELSE null 
        END AS money
    FROM general AS g
    LEFT JOIN active_users AS au ON g.st_id = au.st_id AND g.subject = au.subject
    GROUP BY g.st_id, g.test_grp, g.subject, g.money)

SELECT
agr.test_grp,
agr.subject,
sum(agr.cnt_st_ALL) AS sum_st_ALL,
sum(agr.cnt_st_BUY) AS sum_st_BUY,
sum(agr.cnt_st_ACTIVE) AS sum_st_ACTIVE,
sum(agr.money) AS sum_money
FROM agregated AS agr
GROUP BY agr.test_grp, agr.subject
'''

In [13]:
pd.read_sql(sql, con)

,test_grp,subject,sum_st_ALL,sum_st_BUY,sum_st_ACTIVE,sum_money
0,A,a,2,2.0,1,250.0
1,A,b,2,NaN,1,NaN
2,B,a,1,NaN,0,NaN
3,B,c,1,1.0,1,300.0


In [14]:
# ШАГ 4: произвести требуемые расчеты.

# Предыдущий код уберем в подзапрос "total" с помощью with.
# Все дальнейшие вычисления будут только в основном теле кода.

# Расчитываем необходимые показатели:
    # ARPU (Average Revenue Per User) = sum_money / sum_st_BUY
    # ARPAU (Average Revenue Per Active User) = sum_money / sum_st_ACTIVE
    # CR (Conversion rate) в покупку = sum_st_BUY / sum_st_ALL * 100
    # CR (Conversion rate) активного пользователя в покупку = sum_st_BUY / sum_st_ACTIVE * 100
    # CR пользователя из активности по математике в покупку курса по математике.


sql = '''
with general AS 
    (SELECT
    p.st_id, p.subject, s.test_grp, c.st_id AS st_id_buy, c.sale_time, c.subject AS subject_buy, c.money
    FROM peas_new AS p
    LEFT JOIN studs AS s ON p.st_id = s.st_id
    LEFT JOIN checks AS c ON p.st_id = c.st_id),

active_users AS
    (SELECT
    p.st_id,
    p.subject,
    date(p.timest) AS day,
    sum(CASE
        WHEN (p.correct = 'true' OR p.correct = 1)
        THEN 1 ELSE null
        END) AS sum_correct,
    CASE
        WHEN (sum(CASE
                    WHEN (p.correct = 'true' OR p.correct = 1)
                    THEN 1 ELSE null
                    END) >= 3)
        THEN 1 ELSE null
        END AS active_users
    FROM peas_new as p
    GROUP BY p.st_id, p.subject, day),

agregated AS 
    (SELECT
    g.st_id,
    g.test_grp,
    g.subject,
    COUNT(DISTINCT g.st_id) AS cnt_st_ALL,
    g.subject_buy,
    CASE
        WHEN g.subject = g.subject_buy
        THEN COUNT(DISTINCT g.st_id_buy) ELSE null 
        END AS cnt_st_BUY,
    COUNT(DISTINCT au.active_users) AS cnt_st_ACTIVE,
    CASE
        WHEN g.subject = g.subject_buy
        THEN g.money ELSE null 
        END AS money
    FROM general AS g
    LEFT JOIN active_users AS au ON g.st_id = au.st_id AND g.subject = au.subject
    GROUP BY g.st_id, g.test_grp, g.subject, g.money),

total AS 
    (SELECT
    agr.test_grp,
    agr.subject,
    sum(agr.cnt_st_ALL) AS sum_st_ALL,
    sum(agr.cnt_st_BUY) AS sum_st_BUY,
    sum(agr.cnt_st_ACTIVE) AS sum_st_ACTIVE,
    sum(agr.money) AS sum_money
    FROM agregated AS agr
    GROUP BY agr.test_grp, agr.subject)

SELECT
t.*,
round(t.sum_money * 1.0 / t.sum_st_BUY * 1.0) AS ARPU,
round(t.sum_money * 1.0 / t.sum_st_ACTIVE * 1.0 ) AS ARPAU,
round(t.sum_st_BUY * 1.0 / t.sum_st_ALL * 100.0) AS CR_buy,
round(t.sum_st_BUY * 1.0 / t.sum_st_ACTIVE * 100.0) AS CR_active_buy,
CASE
    WHEN subject = 'math'
    THEN round(sum_st_BUY * 1.0 / sum_st_ACTIVE * 100.0)
    ELSE null
    END AS CR_buy_math
FROM total AS t
'''

In [15]:
pd.read_sql(sql, con)

,test_grp,subject,sum_st_ALL,sum_st_BUY,sum_st_ACTIVE,sum_money,ARPU,ARPAU,CR_buy,CR_active_buy,CR_buy_math
0,A,a,2,2.0,1,250.0,125.0,250.0,100.0,200.0,None
1,A,b,2,NaN,1,NaN,NaN,NaN,NaN,NaN,None
2,B,a,1,NaN,0,NaN,NaN,NaN,NaN,NaN,None
3,B,c,1,1.0,1,300.0,300.0,300.0,100.0,100.0,None


**Итоговый код для изначального условия.**

In [16]:
# ИТОГОВЫЙ КОД

# Заменяем условие активности пользоватея с 3 горошин до 30 горошин/день.


sql = '''
with general AS 
    (SELECT
    p.st_id, p.subject, s.test_grp, c.st_id AS st_id_buy, c.sale_time, c.subject AS subject_buy, c.money
    FROM peas_new AS p
    LEFT JOIN studs AS s ON p.st_id = s.st_id
    LEFT JOIN checks AS c ON p.st_id = c.st_id),

active_users AS
    (SELECT
    p.st_id,
    p.subject,
    date(p.timest) AS day,
    sum(CASE
        WHEN (p.correct = 'true' OR p.correct = 1)
        THEN 1 ELSE null
        END) AS sum_correct,
    CASE
        WHEN (sum(CASE
                    WHEN (p.correct = 'true' OR p.correct = 1)
                    THEN 1 ELSE null
                    END) >= 30)
        THEN 1 ELSE null
        END AS active_users
    FROM peas_new as p
    GROUP BY p.st_id, p.subject, day),

agregated AS 
    (SELECT
    g.st_id,
    g.test_grp,
    g.subject,
    COUNT(DISTINCT g.st_id) AS cnt_st_ALL,
    g.subject_buy,
    CASE
        WHEN g.subject = g.subject_buy
        THEN COUNT(DISTINCT g.st_id_buy) ELSE null 
        END AS cnt_st_BUY,
    COUNT(DISTINCT au.active_users) AS cnt_st_ACTIVE,
    CASE
        WHEN g.subject = g.subject_buy
        THEN g.money ELSE null 
        END AS money
    FROM general AS g
    LEFT JOIN active_users AS au ON g.st_id = au.st_id AND g.subject = au.subject
    GROUP BY g.st_id, g.test_grp, g.subject, g.money),

total AS 
    (SELECT
    agr.test_grp,
    agr.subject,
    sum(agr.cnt_st_ALL) AS sum_st_ALL,
    sum(agr.cnt_st_BUY) AS sum_st_BUY,
    sum(agr.cnt_st_ACTIVE) AS sum_st_ACTIVE,
    sum(agr.money) AS sum_money
    FROM agregated AS agr
    GROUP BY agr.test_grp, agr.subject)

SELECT
t.*,
round(t.sum_money * 1.0 / t.sum_st_BUY * 1.0) AS ARPU,
round(t.sum_money * 1.0 / t.sum_st_ACTIVE * 1.0 ) AS ARPAU,
round(t.sum_st_BUY * 1.0 / t.sum_st_ALL * 100.0) AS CR_buy,
round(t.sum_st_BUY * 1.0 / t.sum_st_ACTIVE * 100.0) AS CR_active_buy,
CASE
    WHEN subject = 'math'
    THEN round(sum_st_BUY * 1.0 / sum_st_ACTIVE * 100.0)
    ELSE null
    END AS CR_buy_math
FROM total AS t
'''

#### Тестирование итогового кода на данных

In [17]:
# Импортируем библиотеки. Создаём подключение к БД.

import pandas as pd
import numpy as np
import sqlite3

con_test = sqlite3.connect('db_test')

In [18]:
df_peas_test = pd.read_csv('https://raw.githubusercontent.com/zhukov-analyst/portfolio/main/sql_data/peas_test.csv', sep=';')

df_studs_test = pd.read_csv('https://raw.githubusercontent.com/zhukov-analyst/portfolio/main/sql_data/studs_test.csv', sep=';')

df_checks_test = pd.read_csv('https://raw.githubusercontent.com/zhukov-analyst/portfolio/main/sql_data/checks_test.csv', sep=';')

In [19]:
df_peas_test['timest'] = pd.to_datetime(df_peas_test['timest'],format='%Y-%m-%d %H:%M:%S')

In [20]:
df_peas_test

,st_id,timest,correct,subject
0,55,2021-03-11 20:32:55,True,geography
1,85,2021-03-11 15:35:19,True,biology
2,39,2021-03-13 10:24:20,True,biology
3,17,2021-03-11 21:53:23,True,geography
4,26,2021-03-13 22:00:12,True,biology
...,...,...,...,...
24995,59,2021-03-13 11:43:29,True,biology
24996,72,2021-03-13 19:33:23,True,biology
24997,6,2021-03-11 05:49:53,True,math
24998,100,2021-03-15 06:48:35,True,algebra


In [21]:
df_studs_test

,st_id,test_grp
0,1,B
1,2,A
2,3,B
3,4,A
4,5,B
...,...,...
95,96,B
96,97,B
97,98,B
98,99,B


In [22]:
df_checks_test

,st_id,sale_time,subject,money
0,1,2021.01.26 11:50:28,biology,597
1,2,2021.01.07 16:18:44,math,530
2,3,2021.02.03 15:47:26,english,694
3,4,2021.01.01 10:30:03,math,132
4,5,2021.02.28 12:13:15,algebra,595
...,...,...,...,...
95,96,2021.01.20 16:59:16,geography,114
96,97,2021.01.01 16:26:42,biology,829
97,98,2021.02.08 12:29:30,math,136
98,99,2021.01.05 18:20:49,algebra,982


In [23]:
# Заливаем таблицу в sql.

df_peas_test.to_sql('peas_test',con_test,index=False,if_exists='replace')

df_studs_test.to_sql('studs_test',con_test,index=False,if_exists='replace')

df_checks_test.to_sql('checks_test',con_test,index=False,if_exists='replace')

100

In [24]:
# ПРОВЕРЯЕМ ИТОГОВЫЙ КОД
# Меняем название таблиц данных:
    # "peas" на "peas_test",
    # "studs" на "studs_test",
    # "checks" на "checks_test".


sql_test = '''
with general AS 
    (SELECT
    p.st_id, p.subject, s.test_grp, c.st_id AS st_id_buy, c.sale_time, c.subject AS subject_buy, c.money
    FROM peas_test AS p
    LEFT JOIN studs_test AS s ON p.st_id = s.st_id
    LEFT JOIN checks_test AS c ON p.st_id = c.st_id),

active_users AS
    (SELECT
    p.st_id,
    p.subject,
    date(p.timest) AS day,
    sum(CASE
        WHEN (p.correct = 'true' OR p.correct = 1)
        THEN 1 ELSE null
        END) AS sum_correct,
    CASE
        WHEN (sum(CASE
                    WHEN (p.correct = 'true' OR p.correct = 1)
                    THEN 1 ELSE null
                    END) >= 30)
        THEN 1 ELSE null
        END AS active_users
    FROM peas_test as p
    GROUP BY p.st_id, p.subject, day),

agregated AS 
    (SELECT
    g.st_id,
    g.test_grp,
    g.subject,
    COUNT(DISTINCT g.st_id) AS cnt_st_ALL,
    g.subject_buy,
    CASE
        WHEN g.subject = g.subject_buy
        THEN COUNT(DISTINCT g.st_id_buy) ELSE null 
        END AS cnt_st_BUY,
    COUNT(DISTINCT au.active_users) AS cnt_st_ACTIVE,
    CASE
        WHEN g.subject = g.subject_buy
        THEN g.money ELSE null 
        END AS money
    FROM general AS g
    LEFT JOIN active_users AS au ON g.st_id = au.st_id AND g.subject = au.subject
    GROUP BY g.st_id, g.test_grp, g.subject, g.money),

total AS 
    (SELECT
    agr.test_grp,
    agr.subject,
    sum(agr.cnt_st_ALL) AS sum_st_ALL,
    sum(agr.cnt_st_BUY) AS sum_st_BUY,
    sum(agr.cnt_st_ACTIVE) AS sum_st_ACTIVE,
    sum(agr.money) AS sum_money
    FROM agregated AS agr
    GROUP BY agr.test_grp, agr.subject)

SELECT
t.*,
round(t.sum_money * 1.0 / t.sum_st_BUY * 1.0) AS ARPU,
round(t.sum_money * 1.0 / t.sum_st_ACTIVE * 1.0 ) AS ARPAU,
round(t.sum_st_BUY * 1.0 / t.sum_st_ALL * 100.0) AS CR_buy,
round(t.sum_st_BUY * 1.0 / t.sum_st_ACTIVE * 100.0) AS CR_active_buy,
CASE
    WHEN subject = 'math'
    THEN round(sum_st_BUY * 1.0 / sum_st_ACTIVE * 100.0)
    ELSE null
    END AS CR_buy_math
FROM total AS t
'''

In [25]:
pd.read_sql(sql_test, con_test)

,test_grp,subject,sum_st_ALL,sum_st_BUY,sum_st_ACTIVE,sum_money,ARPU,ARPAU,CR_buy,CR_active_buy,CR_buy_math
0,A,algebra,46,15,3,7936,529.0,2645.0,33.0,500.0,NaN
1,A,biology,46,7,2,4087,584.0,2044.0,15.0,350.0,NaN
2,A,english,46,10,4,6924,692.0,1731.0,22.0,250.0,NaN
3,A,geography,46,10,7,7162,716.0,1023.0,22.0,143.0,NaN
4,A,math,46,4,5,1555,389.0,311.0,9.0,80.0,80.0
5,B,algebra,54,16,4,10030,627.0,2508.0,30.0,400.0,NaN
6,B,biology,54,9,6,4849,539.0,808.0,17.0,150.0,NaN
7,B,english,54,7,3,4165,595.0,1388.0,13.0,233.0,NaN
8,B,geography,54,13,4,5371,413.0,1343.0,24.0,325.0,NaN
9,B,math,54,9,8,5653,628.0,707.0,17.0,113.0,113.0


**Код работает. Задача решена.**

Код получился громоздким, возможна оптимизация.

Из-за специфики сгенерированных тестовых данных, где оказалось не так много активных пользователей, результаты вычислений "CR_active_buy" немного странные и нелогичные. Помимо этого, остальные результаты вполне интерпритируемы.  


Следующим этапом можно было бы провести статистический анализ различний в конверсии и деньгах между эксперементальными группами A и B.
